In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path
from datetime import datetime

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pyprojroot import here

from knapsack import KnapsackGA
from knapsack.items import items

# Knapsack - Genetic Algorithm

In [ ]:
MAX_WEIGHT = 20
MAX_GENERATIONS = 50

In [ ]:

ga = KnapsackGA(items, max_weight=MAX_WEIGHT, population_size=50)
print(ga)

In [ ]:
solution, result = ga.run(generations=MAX_GENERATIONS)
print(f"{solution=}\n{result.best_fitness=}\n{result.runtime=}s")

These are the items that made the cut:

In [ ]:
print("\n".join([i.name for i in solution]))

## Results and Visualisations

### Create results subdirectory


In [ ]:
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

results_path = here("results") / timestamp
results_path.mkdir(parents=True, exist_ok=True)

In [ ]:
data = pd.DataFrame([record[:-1] for record in result.history], columns=["generation", "best_fitness", "avg_fitness"])

### Plot average population value over time

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
sns.lineplot(data, x="generation", y="best_fitness", ax=ax, label="Best value")
sns.lineplot(data, x="generation", y="avg_fitness", ax=ax, label="Average value")

ax.set(title="Results", xlabel="Generation", ylabel="Value")
ax.legend(loc="upper right", bbox_to_anchor=(1.35, 1))

params = ga.params()
param_text = "Parameters\n\n" + "\n".join(f"{k}: {v}" for k, v in params.items())
ax.text(1.05, 0.8, param_text, transform=ax.transAxes, fontsize=10, verticalalignment='top')
fig.tight_layout()

In [ ]:
filename = "ga_results.png"
filepath = results_path / filename

# Save the figure
fig.savefig(filepath, bbox_inches="tight")

### Plot distributions.

In [ ]:
from knapsack.algorithm import GAResult


def plot_generation_distributions(result: GAResult, generation: int):
    stats = result.history[generation][-1]

    values = [result[0] for result in stats]
    weights = [result[1] for result in stats]

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

    # Plot histogram of values
    ax1.hist(values, bins=15, color='lightgreen', edgecolor='black', alpha=0.7)
    ax1.set_title('Value Distribution')
    ax1.set_xlabel('Total Value')
    ax1.set_ylabel('Number of Solutions')

    max_value = max(values)
    ax1.axvline(x=max_value, color='green', linestyle='--', linewidth=2)
    ax1.annotate(f'Best: {max_value:.1f}', 
                    xy=(max_value, ax1.get_ylim()[1]*0.9),
                    xytext=(max_value*0.75, ax1.get_ylim()[1]*0.9),
                    arrowprops=dict(arrowstyle='->'))

    # Plot histogram of weights
    ax2.hist(weights, bins=15, color='skyblue', edgecolor='black', alpha=0.7)
    ax2.set_title('Weight Distribution')
    ax2.set_xlabel('Total Weight')
    ax2.set_ylabel('Number of Solutions')

    ax2.axvline(x=MAX_WEIGHT, color='red', linestyle='--', linewidth=2)
    ax2.annotate(f'Capacity: {MAX_WEIGHT}', 
                    xy=(MAX_WEIGHT, ax2.get_ylim()[1]*0.9),
                    xytext=(MAX_WEIGHT*1.1, ax2.get_ylim()[1]*0.9),
                    arrowprops=dict(arrowstyle='->'))

    fig.suptitle(f"Generation {generation}")
    plt.tight_layout()

    plt.close(fig)
    return fig    

In [ ]:
generation_to_plot = 0
fig = plot_generation_distributions(result, generation_to_plot)
filename = f"{generation_to_plot}_generation_distribution.png"
filepath = results_path / filename
fig.savefig(filepath, bbox_inches="tight")

In [ ]:
generation_to_plot = MAX_GENERATIONS
fig = plot_generation_distributions(result, generation_to_plot)
filename = f"{generation_to_plot}_generation_distribution.png"
filepath = results_path / filename
fig.savefig(filepath, bbox_inches="tight")

In [351]:
for record in result.history:
    print(record[0], record[-1])

0 [(0.0, 42.01), (0.0, 33.11), (0.0, 25.31), (0.0, 30.41), (0.0, 24.31), (0.0, 27.0), (0.0, 26.6), (0.0, 25.71), (0.0, 32.11), (1069, 20.0), (0.0, 22.7), (0.0, 29.61), (0.0, 35.7), (0.0, 33.11), (0.0, 29.4), (0.0, 31.7), (0.0, 31.8), (0.0, 31.91), (0.0, 24.5), (0.0, 30.11), (0.0, 28.71), (0.0, 23.41), (0.0, 27.11), (0.0, 33.21), (1065, 19.8), (0.0, 26.1), (0.0, 26.8), (0.0, 39.9), (0.0, 27.21), (0.0, 24.91), (0.0, 33.91), (0.0, 31.4), (1221, 19.31), (0.0, 28.71), (0.0, 32.81), (0.0, 36.5), (0.0, 38.3), (0.0, 38.61), (0.0, 43.2), (0.0, 35.1), (0.0, 29.5), (0.0, 33.71), (0.0, 28.61), (0.0, 26.5), (0.0, 31.6), (0.0, 26.91), (0.0, 23.41), (0.0, 34.910000000000004), (0.0, 25.3), (0.0, 31.61)]
1 [(1221, 19.31), (1069, 20.0), (1065, 19.8), (0.0, 42.01), (0.0, 33.11), (1043, 18.4), (0.0, 24.61), (0.0, 29.81), (0.0, 23.3), (0.0, 28.21), (0.0, 34.6), (0.0, 35.0), (0.0, 31.81), (0.0, 33.21), (0.0, 36.8), (0.0, 35.21), (0.0, 33.21), (0.0, 35.9), (1206, 16.91), (0.0, 30.41), (0.0, 35.3), (0.0, 29.7